In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os

# Agregar la carpeta raíz del proyecto al path
sys.path.append(os.path.dirname(os.getcwd()))

In [2]:
import pandas as pd
print("Cargando datos procesados...")
train_df = pd.read_csv('../temp/train_processed.csv')
test_df = pd.read_csv('../temp/test_processed.csv')

# 2. Separar X e y
X_train = train_df.drop(columns=['popularity'])
y_train = train_df['popularity']

X_test = test_df.drop(columns=['popularity'])
y_test = test_df['popularity']

print(f"X_train: {X_train.shape}")
print(f"X_test: {X_test.shape}")

Cargando datos procesados...
X_train: (333345, 43)
X_test: (83336, 43)


In [3]:
from helpers.trainer import ModelTrainer
trainer = ModelTrainer(
    X_train=X_train,
    X_test=X_test,
    y_train=y_train,
    y_test=y_test,
    target_col='popularity'
)

✓ ModelTrainer inicializado - Target: popularity
  Train: 333,345 muestras, 43 features
  Test: 83,336 muestras
  Target range: [0.0, 96.0]
  Target mean: 29.3 ± 15.8


In [ ]:
custom_grid = {
    'n_estimators': [200, 300],
    'learning_rate': [0.05, 0.1],
    'max_depth': [5, 7]
} # Pueden configurar esta grid como deseen o quitarla para usar la por defecto

In [4]:
trainer.train_model(
    model_name='lightgbm',
    cv=5,
    param_grid=None, # Pueden pasar custom_grid o None para usar la grid por defecto
)


ENTRENANDO: LIGHTGBM
Grid de búsqueda:
  n_estimators: [100, 200, 300]
  learning_rate: [0.01, 0.05, 0.1]
  max_depth: [5, 7, 10]
  num_leaves: [31, 50]
  min_child_samples: [20, 30]
  subsample: [0.8, 1.0]
  colsample_bytree: [0.8, 1.0]

 Buscando mejores hiperparámetros con 5-Fold CV...
  Total combinaciones: 432

  [1/432] Probando: {'n_estimators': 100, 'learning_rate': 0.01, 'max_depth': 5, 'num_leaves': 31, 'min_child_samples': 20, 'subsample': 0.8, 'colsample_bytree': 0.8}
    MSE CV: 205.4502 (±1.3874)
    ✓ Nuevo mejor score: 205.4502

  [2/432] Probando: {'n_estimators': 100, 'learning_rate': 0.01, 'max_depth': 5, 'num_leaves': 31, 'min_child_samples': 20, 'subsample': 0.8, 'colsample_bytree': 1.0}
    MSE CV: 204.0837 (±1.3820)
    ✓ Nuevo mejor score: 204.0837

  [3/432] Probando: {'n_estimators': 100, 'learning_rate': 0.01, 'max_depth': 5, 'num_leaves': 31, 'min_child_samples': 20, 'subsample': 1.0, 'colsample_bytree': 0.8}
    MSE CV: 205.4502 (±1.3874)

  [4/432] Proban

{'model_name': 'lightgbm',
 'best_params': {'n_estimators': 300,
  'learning_rate': 0.1,
  'max_depth': 7,
  'num_leaves': 50,
  'min_child_samples': 30,
  'subsample': 0.8,
  'colsample_bytree': 1.0},
 'cv_score': np.float64(151.16935095957373),
 'train_metrics': {'mse': 142.29158750628068,
  'rmse': np.float64(11.928603753427334),
  'mae': 9.432999129841805,
  'r2': 0.4273767434946051},
 'test_metrics': {'mse': 152.28775880979853,
  'rmse': np.float64(12.340492648585734),
  'mae': 9.721218494106246,
  'r2': 0.3893160633456746},
 'training_time': 3024.028849,
 'model': LGBMRegressor(max_depth=7, min_child_samples=30, n_estimators=300, n_jobs=-1,
               num_leaves=50, random_state=42, subsample=0.8, verbose=-1)}

In [5]:
print(trainer.results.keys())  #Debe incluir el modelo entrenado

dict_keys(['lightgbm'])


In [ ]:
results = trainer.results['lightgbm'] #No da error si el modelo fue entrenado correctamente
print(f"LightGBM CV MSE: {results['cv_score']:.4f}")
print(f"LightGBM Test R²: {results['test_metrics']['r2']:.4f}")

XGBoost CV MSE: 151.1694
XGBoost Test R²: 0.3893


In [9]:
trainer.save_model(model_name='lightgbm', path='../models/lightgbm_model.joblib')

✓ Modelo guardado: ../models/lightgbm_model.joblib
✓ Metadata guardada: ../models/lightgbm_model_metadata.joblib


In [13]:
from helpers.trainer import load_model
# Para cargar el modelo guardado posteriormente
loaded_model,metadata = load_model(path='../models/lightgbm_model.joblib')

loaded_model


✓ Modelo: lightgbm
  CV MSE: 151.1694
  Test R²: 0.3893
  Test RMSE: 12.3405


LGBMRegressor(max_depth=7, min_child_samples=30, n_estimators=300, n_jobs=-1,
              num_leaves=50, random_state=42, subsample=0.8, verbose=-1)